# Auto Grading and Review Generator
This file evaluate students' homeworks using AI assistant and generate reviews automatically.

## 1. Introduce AI tool first

In [ ]:
# If use ollama, no API_KEY needed
%load_ext jupyter_ai_magics
%config AiMagics.default_language_model = 'ollama:phi4'

In [ ]:
# If use Gemini, have to load .env file to get API_KEY
%load_ext jupyter_ai_magics
%config AiMagics.default_language_model = 'gemini:gemini-2.0-flash-thinking-exp'

from dotenv import load_dotenv

load_dotenv()

## 2. Use AI to evaluate the given code
Three main steps:
1. Get all students' homeworks by iterate the homework folder
2. Generate prompts for AI to give review and score to each homework
3. Gathering all the output reviews and write them to specific notebooks in the review folder

### 2.0 Define the review parametrers

In [9]:
keyword = '----Answer----'  # keyword to split the question and answer, should be predefined
root = './homework'         # root folder to store students' homework, file name should be predefined
active = 'lesson2'          # the active lesson, should change according to the lesson you are teaching
review_folder = './review'  # folder to store the review result, should be predefined

### 2.1 Get the homeworks under `root` directory
Define the class to get all homeworks: Iterate the `homework` folder, get all the code section from all students' homeworks according to the active lesson.

In [10]:
import nbformat, os
from nbclient import NotebookClient

class HomeworkManager:
    def __init__(self, key_word, root_dir, active_lesson):
        self.key_word = key_word
        self.root_dir = root_dir
        self.active_lesson = active_lesson
        
    def get_all_homework(self):
        all_homeworks = []
        for dir_path, _, file_names in os.walk(self.root_dir):
            for file_name in file_names:
                # 忽略以点开头的隐藏文件
                if not file_name.startswith('.'): 
                    if file_name.strip().split('.')[0].split('_')[-1] == self.active_lesson:
                        file_path = os.path.join(dir_path, file_name)
                        # print(file_path)
                        all_homeworks.append(file_path)
                        if all_homeworks == []:
                            print(f"There's no homework for {self.active_lesson}!")
                    # else:
                    #     print(f"There's no homework for {self.active_lesson}!")
        return all_homeworks
    
    # def get_cell_content(self, file_path):
    #     for dir_path, _, file_names in os.walk(self.root_dir):
    #         for file_name in file_names:
    #             file_path = os.path.join(dir_path, file_name)
    #             all_homeworks.append(file_path)
    #     return all_homeworks
    
    def get_cell_content(self, file_path):
        with open(file_path) as f:
            nb = nbformat.read(f, as_version=4)
        review_cell = [cell for cell in nb.cells if self.key_word.lower() in cell.source.lower()]
        if len(review_cell) == 0:
            raise IndexError("There's no answer in the notebook.")
        else:
            return review_cell

### 2.2 Generate prompts for each homework
Define a class to create cells in a notebook.

In [11]:
import time
from IPython.core.getipython import get_ipython

class NotebookManager:
    def __init__(self):
        self.shell = get_ipython()

    def create_code_cell(self, content, cell_type='code'):
        """Create new code cell"""
        payload = dict(
            source = 'set_next_input',
            text = content,
            replace = False
        )
        self.shell.payload_manager.write_payload(payload)

    def create_markdown_cell(self, content, cell_type='markdown'):
        """Create new markdown cell"""
        payload = dict(
            source = 'set_next_input',
            text = content,
            replace = False
        )
        self.shell.payload_manager.write_payload(payload)

    def run_cell(self, code):
        result = self.shell.run_cell(code)
        return result
    
    def create_and_run(self, content):
        # self.create_code_cell(content)
        # time.sleep(0.5)
        # return self.run_cell(content)
        # 添加输出追踪
        # self.last_output = None
        return self.run_cell(content)
        # result = self.run_cell(content)
        # # 保存输出结果
        # if hasattr(result, 'result'):
        #     self.last_output = result.result
        # return result

### 2.3 Let's generate reviews automatically!!
1. Iterate all homeworks
2. Extract activate lesson, name and code section to review
3. Generate prompt for each code section
4. Gether and store every review message from AI reviewer
5. Write to new cell in new notebook, and store them in given folder

In [ ]:
hws = HomeworkManager(keyword, root, active)

homeworks = hws.get_all_homework()
reviews_array = []

nb = NotebookManager()

for homework in homeworks:
    file_name = os.path.basename(homework)
    # Extract the main body of file name, discard the extension name
    file_name = file_name.split('.')[0]
    lesson_idx = file_name.split('_')[-1]
    stu_name = file_name.split('_')[0]
    code_seg = hws.get_cell_content(homework)[0].source
    # print(hw_dict['name'])
    prompt = "%%ai\n"
    prompt += "Main task: {stu_name} is a beginner in Python coding. Please review his(her) code of {lesson_idx} shown in {code_seg}, rated them in 5-score system and give some advice from the perspective of {stu_name}'s teacher, omit advanced coding skills."
    prompt += "Requirements: 1.Use Chinese to format your answer. 2.Your review should start with 'Review:{stu_name}:{lesson_idx}'. 3.When you finish reviewing, add a seperator line and write a EOF marker."

    result = nb.create_and_run(prompt)
    if hasattr(result.result, '_repr_markdown_'):
        reviews_array.append(result.result._repr_markdown_())
        print("GOT YOU!")
    # nb.create_code_cell(prompt)

In [ ]:
# Remove title and tail lines
def remove_lines(text, head=1, tail=1):
    lines = text.splitlines()
    if not head < 0:
        if not tail < 0:
            return '\n'.join(lines[head:-(tail+1)])


# `reviews_array` is a tuple variable
for review in reviews_array:
    if review[0].startswith("```"):
        # Get the real first line: Reivew:Name
        first_line = review[0].split('\n')[1]
        # Extract lesson idx and student name in the text. <<< Review:name:lesson >>>
        # Get the active lesson index
        lesson = first_line.split(':')[-1]
        # Get the student name
        name = first_line.split(':')[-2].upper().strip()
        # Remove the first and last line
        text = remove_lines(review[0], 1, 1)

        # Create a new notebook
        new_nb = nbformat.v4.new_notebook()
        # Add markdown cell
        new_nb.cells.append(nbformat.v4.new_markdown_cell(text))

        # Save Notebook
        output_path = f'./reviews/review_{name}_{lesson}.ipynb'
        os.makedirs('./reviews', exist_ok=True)
        
        with open(output_path, 'w', encoding='utf-8') as f:
            nbformat.write(new_nb, f)
        print(f"{name}'s review has been created：{output_path}")